In [ ]:
!pip3 uninstall -y tensorflow
!pip3 install tensorflow==1.15

In [1]:
cd drive/My Drive/dl/cose474/assignment5/

/content/drive/My Drive/dl/cose474/assignment5


In [2]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [3]:
import os
import time
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np


In [4]:
class GAN(object):

    def __init__(self, config):
        self.D_hidden_dim = config["D_hidden_dim"]
        self.G_hidden_dim = config["G_hidden_dim"]
        self.Z_dim = config["Z_dim"]

        # Placeholders for input
        self.input_x = tf.placeholder(tf.float32, shape=[None, 784])
        self.input_z = tf.placeholder(tf.float32, shape=[None, self.Z_dim])

        self.D_W1 = tf.get_variable("D_W1", shape=[784, self.D_hidden_dim], initializer=tf.contrib.layers.xavier_initializer())
        self.D_b1 = tf.Variable(tf.zeros(shape=[self.D_hidden_dim]))

        self.D_W2 = tf.get_variable("D_W2", shape=[self.D_hidden_dim, 1], initializer=tf.contrib.layers.xavier_initializer())
        self.D_b2 = tf.Variable(tf.zeros(shape=[1]))

        self.theta_D = [self.D_W1, self.D_W2, self.D_b1, self.D_b2]

        self.G_W1 = tf.get_variable("G_W1", shape=[self.Z_dim, self.G_hidden_dim], initializer=tf.contrib.layers.xavier_initializer())
        self.G_b1 = tf.Variable(tf.zeros(shape=[self.G_hidden_dim]))

        self.G_W2 = tf.get_variable("G_W2", shape=[self.G_hidden_dim, 784], initializer=tf.contrib.layers.xavier_initializer())
        self.G_b2 = tf.Variable(tf.zeros(shape=[784]))

        self.theta_G = [self.G_W1, self.G_W2, self.G_b1, self.G_b2]

        self.G_sample = self.generator(self.input_z)
        D_real, D_logit_real = self.discriminator(self.input_x)
        D_fake, D_logit_fake = self.discriminator(self.G_sample)

        D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
        D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
        self.D_loss = D_loss_real + D_loss_fake
        self.G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

    def sample_Z(self, m, n):
        return np.random.uniform(-1., 1., size=[m, n])

    def generator(self, z):
        G_h1 = tf.nn.relu(tf.matmul(z, self.G_W1) + self.G_b1)
        G_log_prob = tf.matmul(G_h1, self.G_W2) + self.G_b2
        G_prob = tf.nn.sigmoid(G_log_prob)
        return G_prob

    def discriminator(self, x):
        D_h1 = tf.nn.relu(tf.matmul(x, self.D_W1) + self.D_b1)
        D_logit = tf.matmul(D_h1, self.D_W2) + self.D_b2
        D_prob = tf.nn.sigmoid(D_logit)
        return D_prob, D_logit

In [6]:
tf.flags.DEFINE_string('f', '', 'kernel')
def del_all_flags(FLAGS):
  flags_dict = FLAGS._flags()
  keys_list = [keys for keys in flags_dict]
  for keys in keys_list:
    FLAGS.__delattr__(keys)


In [7]:
del_all_flags(tf.flags.FLAGS)

In [5]:
# Model Hyperparameters
tf.flags.DEFINE_integer("Z_dim", 100, "Dimensionality of noise vector (default: 100)")
tf.flags.DEFINE_integer("G_hidden_dim", 128, "Dimensionality of hidden layer (default: 100)")
tf.flags.DEFINE_integer("D_hidden_dim", 128, "Dimensionality of hidden layer (default: 100)")

tf.flags.DEFINE_float("lr", 1e-3, "Learning rate(default: 0.01)")
tf.flags.DEFINE_float("lr_decay", 0.99, "Learning rate decay rate (default: 0.98)")
# Training parameters
tf.flags.DEFINE_integer("batch_size", 100, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 500, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 500, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 3, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS

def preprocess():
    # Load data
    mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)
    print("Train: {:d}".format(mnist.train.num_examples))
    return mnist

def train(mnist):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            gan = GAN(FLAGS.flag_values_dict())

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            decayed_lr = tf.train.exponential_decay(FLAGS.lr, global_step, 1000, FLAGS.lr_decay, staircase=True)
            D_solver = tf.train.AdamOptimizer(decayed_lr).minimize(gan.D_loss, var_list=gan.theta_D)
            G_solver = tf.train.AdamOptimizer(decayed_lr).minimize(gan.G_loss, var_list=gan.theta_G, global_step=global_step)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            if not os.path.exists('out/'):
                os.makedirs('out/')

            for epoch in range(FLAGS.num_epochs):
                total_batch = int(mnist.train.num_examples / FLAGS.batch_size)

                for _ in range(total_batch):
                    batch_xs, batch_ys = mnist.train.next_batch(FLAGS.batch_size)

                    _, D_loss_curr = sess.run([D_solver, gan.D_loss], feed_dict={gan.input_x: batch_xs, gan.input_z: gan.sample_Z(FLAGS.batch_size, FLAGS.Z_dim)})
                    _, step, G_loss_curr = sess.run([G_solver, global_step, gan.G_loss], feed_dict={gan.input_z: gan.sample_Z(FLAGS.batch_size, FLAGS.Z_dim)})
                    if step % 100 == 0:

                        samples, step = sess.run([gan.G_sample, global_step],
                                                 feed_dict={gan.input_z: gan.sample_Z(16, FLAGS.Z_dim)})

                        fig = plot(samples)
                        plt.savefig('out/{}.png'.format(str(step).zfill(3)), bbox_inches='tight')
                        plt.close(fig)
                        saver.save(sess, checkpoint_prefix, global_step=step)
                        print('Iter: {}'.format(step))
                        print('D loss: {:.4}'.format(D_loss_curr))
                        print('G_loss: {:.4}'.format(G_loss_curr))
                        print()

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

def main(argv=None):
    mnist = preprocess()
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


W1008 10:07:37.532809 140195636123520 deprecation.py:323] From <ipython-input-5-ececab1df23b>:22: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please write your own downloading logic.


W1008 10:07:37.534503 140195636123520 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please use urllib or similar directly.


...

Writing to /content/drive/My Drive/dl/cose474/assignment5/runs/1602151660

Iter: 100
D loss: 0.2829
G_loss: 3.575

Iter: 200
D loss: 0.1794
G_loss: 4.882

Iter: 300
D loss: 0.05992
G_loss: 5.326

Instructions for updating:
Use standard file APIs to delete files with this prefix.


W1008 10:07:45.882351 140195636123520 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/saver.py:963: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


Iter: 400
D loss: 0.01348
G_loss: 5.932

Iter: 500
D loss: 0.02088
G_loss: 5.852

Iter: 600
D loss: 0.02731
G_loss: 5.549

Iter: 700
D loss: 0.009674
G_loss: 6.437

Iter: 800
D loss: 0.04026
G_loss: 8.378

Iter: 900
D loss: 0.01217
G_loss: 6.254

Iter: 1000
D loss: 0.02253
G_loss: 4.793

Iter: 1100
D loss: 0.02234
G_loss: 5.827

Iter: 1200
D loss: 0.0377
G_loss: 5.345

Iter: 1300
D loss: 0.02032
G_loss: 5.801

Iter: 1400
D loss: 0.02446
G_loss: 6.319

Iter: 1500
D loss: 0.008068
G_loss: 6.174

Iter: 1600
D loss: 0.02287
G_loss: 5.651

Iter: 1700
D loss: 0.07042
G_loss: 5.178

Iter: 1800
D loss: 0.09765
G_loss: 6.071

Iter: 1900
D loss: 0.03908
G_loss: 5.688

Iter: 2000
D loss: 0.02422
G_loss: 5.484

Iter: 2100
D loss: 0.1161
G_loss: 4.356

Iter: 2200
D loss: 0.09316
G_loss: 4.982

Iter: 2300
D loss: 0.07856
G_loss: 6.023

Iter: 2400
D loss: 0.04476
G_loss: 4.476

Iter: 2500
D loss: 0.109
G_loss: 4.172

Iter: 2600
D loss: 0.1242
G_loss: 4.01

Iter: 2700
D loss: 0.1171
G_loss: 4.985

Ite

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
